In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import cv2
import mediapipe as mp
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('fer2013.csv')

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

data = []
labels = []

for idx, row in df.iterrows():
    pixels = np.fromstring(row['pixels'], sep=' ').reshape(48, 48).astype(np.uint8)
    image = cv2.cvtColor(pixels, cv2.COLOR_GRAY2BGR)
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if results.multi_face_landmarks:
        for landmarks in results.multi_face_landmarks:
            landmark_points = np.array([[lm.x, lm.y] for lm in landmarks.landmark])
            data.append(landmark_points.flatten())
            labels.append(row['emotion'])

    if idx % 1000 == 0:
        print(f'Processed {idx} images')

np.save('data.npy', np.array(data))
np.save('labels.npy', np.array(labels))

In [8]:
data_path = '/content/drive/MyDrive/data.npy'
label_path = '/content/drive/MyDrive/labels.npy'

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [14]:
from tensorflow.keras import models, layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Load landmark data
X = np.load(data_path)  # shape: (samples, 936)
y = np.load(label_path)

le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_categorical = to_categorical(y_encoded, num_classes=7)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Reshape for Transformer: (samples, seq_len, 1)
X_train = X_train.reshape((-1, 936, 1))
X_test = X_test.reshape((-1, 936, 1))

# Transformer Encoder Block
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(key_dim=head_size, num_heads=num_heads, dropout=dropout)(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Dense(ff_dim, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    return x + res

# Model Input
inputs = layers.Input(shape=(936, 1))

x = inputs
for _ in range(3):
    x = transformer_encoder(x, head_size=64, num_heads=4, ff_dim=128, dropout=0.2)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(7, activation="softmax")(x)

# Compile Model
model = models.Model(inputs, outputs)
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(learning_rate=0.0005), metrics=["accuracy"])

# Train
early_stop = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(patience=3, factor=0.5, verbose=1)

model.fit(X_train, y_train, validation_data=(X_test, y_test),
          epochs=50, batch_size=128, callbacks=[early_stop, reduce_lr])

model.save('emotion_transformer_final.h5')


AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

# Load model
model = tf.keras.models.load_model('emotion_transformer_final.h5')
class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Mediapipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

# Webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img_rgb)

    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0].landmark
        landmark_list = []
        for lm in landmarks:
            landmark_list.extend([lm.x, lm.y])

        landmark_array = np.array(landmark_list).reshape(1, 936, 1)
        prediction = model.predict(landmark_array, verbose=0)
        emotion = class_names[np.argmax(prediction)]

        # Show emotion label
        cv2.putText(frame, f'Emotion: {emotion}', (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)

    cv2.imshow('Real-time Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
